<a href="https://colab.research.google.com/github/ainfanzon/Cockroach_IAM_Workshop/blob/main/Exercise_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

### CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

# Identity and Access Management Workshop.
---
## Multiregional Support.

CockroachDB's distributed architecture allows it to replicate data across multiple nodes in different [geographic regions](https://www.cockroachlabs.com/docs/stable/multiregion-overview), ensuring that data is available even if one or more nodes or entire regions become unavailable.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=14Mm3_GcslyAkF2pHiD2tbsPsZLMLQQxP" width="800" height="345">
      </td>
  </tr>
</table>

</body>
</html>

In this section you will:

1. Learn muliregion concepts.
1. Launch a nine nodes cluster.
1. Create and load the **movr** database.
1. Convert it to a multiregion database.
1. Configure regional by row.
1. Setting Regional Survival Goal

---

## 1. Muliregion Concepts.

There are three basic concepts in a multiregion deployment, namely:

- Region - A geographical location that has a set of CockroachDB nodes deployed within it. One of the regions is defined as *primary* and it is the default for all the tables in the database.
- Zones - A region has one or more zoness. Zones are typically aligned with data centers within a region.
- Survival goals - In a multi-region deployment are zone and region survival:
  - Zone survival - Is the default setting. It ensures that the database remains available for reads and writes if a zone becomes unavailable.
  - Region survival - Ensures that the database remains available for reads and writes even if an entire region becomes unavailable. This comes at the cost of increased write latency, but read performance is unaffected.
- Locality - distribution within a table to optimize access from specific regions
  - Global table - optimized fro low-latency reads from any region
  - Regional table - Optimized for low-latency reads and writes from a single region.
  - Regional by row - optimized for low-latency reads and writes for a region. Rows are assigned to specific regions.

---

## 2. Launch a nine nodes cluster.

- Copy and paste the following statements to your terminal.

> <code>
cockroach start --insecure --listen-addr=10.0.1.2:26257 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8080 --locality=region=us-west,zone=us-west-1a --store=/home/cockroach/data/cr_data_1 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26258 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8081 --locality=region=us-west,zone=us-west-1b --store=/home/cockroach/data/cr_data_2 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26259 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8082 --locality=region=us-west,zone=us-west-1c --store=/home/cockroach/data/cr_data_3 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26260 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8083 --locality=region=us-east,zone=us-east-1a --store=/home/cockroach/data/cr_data_4 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26261 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8084 --locality=region=us-east,zone=us-east-1b --store=/home/cockroach/data/cr_data_5 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26262 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259 --http-addr=10.0.1.2:8085 --locality=region=us-east,zone=us-east-1c --store=/home/cockroach/data/cr_data_6 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26263 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259,10.0.1.2:26260,10.0.1.2:26261,10.0.1.2:26262 --http-addr=10.0.1.2:8086 --locality=region=europe-west,zone=europe-west1a --store=/home/cockroach/data/cr_data_7 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26264 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259,10.0.1.2:26260,10.0.1.2:26261,10.0.1.2:26262 --http-addr=10.0.1.2:8087 --locality=region=europe-west,zone=europe-west1b --store=/home/cockroach/data/cr_data_8 --background<br><br>
cockroach start --insecure --listen-addr=10.0.1.2:26265 --join=10.0.1.2:26257,10.0.1.2:26258,10.0.1.2:26259,10.0.1.2:26260,10.0.1.2:26261,10.0.1.2:26262 --http-addr=10.0.1.2:8088 --locality=region=europe-west,zone=europe-west1c --store=/home/cockroach/data/cr_data_9 --background<br><br>
cockroach init --insecure --host 10.0.1.2
</code>

The above commands will launch a multiregion deployment depicted below:
<br>
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1YQ55Zp8KLy_ae7REkHG3TybeHU2KUDwk" width="850" height="450">
      </td>
  </tr>
</table>

</body>
</html>

- Verify the cluster deployment.

> <cose>
pgrep -a cockroach | awk '{ print $5}'<br><br>
--listen-addr=10.0.1.2:26257<br>
--listen-addr=10.0.1.2:26258<br>
--listen-addr=10.0.1.2:26261<br>
--listen-addr=10.0.1.2:26262<br>
--listen-addr=10.0.1.2:26259<br>
--listen-addr=10.0.1.2:26260<br>
--listen-addr=10.0.1.2:26263<br>
--listen-addr=10.0.1.2:26264<br>
--listen-addr=10.0.1.2:26265<br>
</code>

- Open another browser tab to display the cockroach DB Console http://35.235.99.125:8080/#/overview/list.

---

## 3. Create and load the **movr** database.

<br>
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
        <img  align="left" src="https://drive.google.com/uc?id=1RBFyaB6DPB1_tPrkAnGuCnd6ShizXZ69" width="525" height="325">
      </td>
  </tr>
</table>
</body>
</html>
<br>

To create the **movr** database execute the command below:<br><br>

**NOTE: Don't forget to check your Python web server is running on port 3000**

> <code>cockroach sql --host 10.0.1.2 -u root -d default -f /home/cockroach/sql/movr.sql --insecure
</code>


Connect to the database

In [ ]:
import psycopg2
import pandas as pd
from pandas import Series

from IPython.display import IFrame, display, HTML, Markdown

pd.set_option('display.max_colwidth', None)

conn = psycopg2.connect(
        database = 'movr'
      , user = 'root'
      , host = '35.235.99.125'                        # Use the GCP Compute Engine external IP address
      , port = '26257'
      , sslmode = 'disable'
)
cursor = conn.cursor()


<html>
<body>
  <div class="container">
    <div class="image">
      <img style=”float: right; padding: 0px 0px 0px 3px;” align="right" src="https://drive.google.com/uc?id=1DvrZ4ZJpxFgFxgAuAnB-RN0WapuT_54u" width="450" height="210">
    </div>
    <div style="text-align: center; margin-top: 5px; padding: 13px 0 10px 0;">
      <p>Execute the <code>SHOW REGIONS;</code> command to display:
      <br>
      <ul>
        <li>The name of the region.</li>
        <li>The availability zones for the region.</li>
        <li>A set of database names that use the region.</li>
        <li>A set of database names for which the region is the primary region.</li>
        <li>A set of database names for which the region is the secondary region.</li>
      </.ul>
    </div>
  </div>
</body>
</html>

In [ ]:
cursor.execute("""
SHOW REGIONS;
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set = df_result_set.reset_index(drop=True)
df_result_set

,region,zones,database_names,primary_region_of,secondary_region_of
0,europe-west,"[europe-west1a, europe-west1b, europe-west1c]",[],[],[]
1,us-east,"[us-east-1a, us-east-1b, us-east-1c]",[],[],[]
2,us-west,"[us-west-1a, us-west-1b, us-west-1c]",[],[],[]


- In which region is the **movr** database located?
- How many _zones_ does each _region_ has?

In [ ]:
cursor.execute("""
SELECT raw_config_sql
FROM crdb_internal.zones
WHERE target = 'RANGE default'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
s = pd.Series(df_result_set['raw_config_sql'])
x = s.to_string(index=False, name='raw_config_sql')
print(x.replace(r'\n\t', '\n\t'))

ALTER RANGE default CONFIGURE ZONE USING
	range_min_bytes = 134217728,
	range_max_bytes = 536870912,
	gc.ttlseconds = 14400,
	num_replicas = 3,
	constraints = '[]',
	lease_preferences = '[]'
Name: raw_config_sql


<html>
<body>
  <div class="container">
    <div class="image">
      <img style=”float: right; padding: 0px 0px 0px 3px;” align="right" src="https://drive.google.com/uc?id=1Gejgp5qbI2b4udQRGytEIxS0OvGykiT5" width="450" height="210">
    </div>
    <div style="text-align: center; margin-top: 5px; padding: 13px 0 10px 0;">
      <p>Execute the <code>SHOW RANGES FROM DATABASE movr;</code> command to display:
      <br>
      <ul>
        <li>The start key for the range.</li>
        <li>The end key for the range.</li>
        <li>The nodes that contain the range's replicas.</li>
        <li>The localities of the range's replicas.</li>
        <li>	The nodes that contain the range's voting replicas (that is, the replicas that participate in Raft elections).</li>
      </.ul>
    </div>
  </div>
</body>
</html>

In [ ]:
cursor.execute("""
SELECT range_id, replicas
FROM [SHOW RANGES FROM TABLE users]
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,range_id,replicas
0,69,"[2, 3, 4]"
1,79,"[1, 7, 9]"


In [ ]:
cursor.execute("""
SELECT range_id, replicas, lease_holder
FROM [SHOW RANGES FROM TABLE users WITH DETAILS]
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,range_id,replicas,lease_holder
0,69,"[2, 3, 4]",4
1,79,"[1, 7, 9]",9


- Which nodes contain replicas of the movr database?
- What are the regions and zones? Are they on the same region?

---

## 4. Convert it to a multiregion database.

The SHOW REGIONS statement lists the cluster regions for a multi-region cluster, or the database regions for the databases in a multi-region cluster.

Assign Primary Region and Associate Non-Primary Regions to the **movr** Database

<html>
<body>

<table>
  <tr>
      <td>
          <table>
              <tr>
                  <th>Region</th>
                  <th>Availability Zone</th>          
              </tr>
              <tr>
                  <td>us-east1</td>
                  <td>us-east1a<br>us-east1b<br>us-east1c</td>
              </tr>
              <tr>
                  <td>us-west1</td>
                  <td>us-west1b</td>
              </tr>
              <tr>
                  <td>europe-west1</td>
                  <td>europe-west1c</td>
              </tr>
          </table>
      </td>
      <td>
          <img align="left" src="https://drive.google.com/uc?id=18ZUG07gpni11RISjm6jv11sK7_k6DR2x" width="500" height="300">
      </td>
  </tr>
</table>

</body>
</html>
<br>
To convert the movr database to multiregion, you need to assign a primary region and associate the nonprimary regions with the database.

- A multi-region deployment requires an enterprise license.

> <code>
cockroach sql --insecure --host=10.0.1.2:26257 -d defaultdb -f /home/cockroach/sql/apply_lic.sql<br>

- Associate the database to a primary and secondary regions.
<br>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d movr --execute="ALTER DATABASE movr SET PRIMARY REGION= 'us-west';"<br>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d movr --execute="ALTER DATABASE movr ADD REGION 'us-east';"<br>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d movr --execute="ALTER DATABASE movr ADD REGION 'europe-west';"
</code>

In [ ]:
cursor.execute("""
SELECT name, locality
FROM crdb_internal."tables"
WHERE schema_name='public'
AND database_name = 'movr'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,name,locality
0,promo_codes,None
1,users,None
2,user_promo_codes,None
3,vehicles,None
4,rides,None
5,vehicle_location_histories,None


In [ ]:
cursor.execute("""
ALTER DATABASE movr SET PRIMARY REGION= "us-west"
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

SerializationFailure: restart transaction: TransactionRetryWithProtoRefreshError: WriteTooOldError: write for key /Table/3/1/106/2/1 at timestamp 1731004516.916504171,0 too old; must write at or above 1731005074.051451065,2: "sql txn" meta={id=4ca93b33 key=/Table/3/1/106/2/1 iso=Serializable pri=0.00399849 epo=0 ts=1731005074.051451065,2 min=1731004516.916504171,0 seq=6} lock=true stat=PENDING rts=1731004516.916504171,0 wto=false gul=1731004517.416504171,0
HINT:  See: https://www.cockroachlabs.com/docs/v24.2/transaction-retry-error-reference.html


## 5. Configure Regional by row

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td>
          <img align="left" src="https://drive.google.com/uc?id=1NxDSdkeSOiBbpR7O40IdCufzPXE8Vs8d" width="500" height="300">
      </td>
      <td>
          <table style="width:100%">
              <tr>
                  <th>Region</th>
                  <th>Row Location</th>          
              </tr>
              <tr>
                  <td>us-east1</td>
                  <td>new york<br>boston<br>washington dc<br><br><em>others</em></td>
              </tr>
              <tr>
                  <td>us-west1</td>
                  <td>san francisco<br>seattle<br>los angeles</td>
              </tr>
              <tr>
                  <td>europe-west1</td>
                  <td>amsterdam<br>paris<br>rome</td>
              </tr>
          </table>
      </td>
  </tr>
</table>
    
Assign rows to regions based on a given value


In [ ]:
cursor.execute("""
ALTER TABLE users ADD COLUMN crdb_region crdb_internal_region AS (
    CASE
        WHEN city in ('new york', 'boston', 'washington dc') THEN 'gcp-us-east1'
        WHEN city in ('san francisco', 'seattle', 'los angeles') THEN 'gcp-us-west1'
        WHEN city in ('amsterdam', 'paris', 'rome') THEN 'gcp-europe-west1'
        ELSE 'gcp-us-east1'
    END
) STORED
""")
conn.commit()

DuplicateColumn: column "crdb_region" of relation "customers" already exists


In [ ]:
cursor.execute("""
SELECT name AS "Table Name"
     , locality AS "Locality"
FROM crdb_internal."tables"
WHERE schema_name='public'
AND database_name = 'northwind'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.set_index('Table Name', inplace=True)
df_result_set

,Locality
Table Name,


In [ ]:
cursor.execute("""
SHOW regions
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,region,zones,database_names,primary_region_of,secondary_region_of
0,europe-west,"[europe-west1a, europe-west1b, europe-west1c]",[],[],[]
1,us-east,"[us-east-1a, us-east-1b, us-east-1c]",[],[],[]
2,us-west,"[us-west-1a, us-west-1b, us-west-1c]",[northwind],[northwind],[]


In [ ]:
cursor.execute("""
SELECT DISTINCT crdb_region
     , city
FROM customers
ORDER BY crdb_region
""")
conn.commit()
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,crdb_region,city
0,europe-west,Genève
1,europe-west,Strasbourg
2,europe-west,London
3,europe-west,Stuttgart
4,europe-west,Mannheim
...,...,...
64,us-west,Elgin
65,us-west,Anchorage
66,us-west,Seattle
67,us-west,Portland



> <code>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d northwind --execute="ALTER TABLE customers ALTER COLUMN crdb_region SET NOT NULL;"<br>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d northwind --execute="ALTER TABLE customers SET LOCALITY REGIONAL BY ROW;"<br>
cockroach sql --insecure --host=10.0.1.2:26257 -u root -d northwind --execute="SELECT job_type, description, status FROM [show jobs] WHERE description LIKE '%REGIONAL%';"

In [ ]:
cursor.execute("""
SELECT name, locality
FROM crdb_internal."tables"
WHERE schema_name='public'
AND database_name = 'northwind'
""")
conn.commit()
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set

,name,locality
0,categories,REGIONAL BY TABLE IN PRIMARY REGION
1,customer_customer_demo,REGIONAL BY TABLE IN PRIMARY REGION
2,customer_demographics,REGIONAL BY TABLE IN PRIMARY REGION
3,customers,REGIONAL BY ROW
4,employees,REGIONAL BY TABLE IN PRIMARY REGION
5,employee_territories,REGIONAL BY TABLE IN PRIMARY REGION
6,order_details,REGIONAL BY TABLE IN PRIMARY REGION
7,orders,REGIONAL BY TABLE IN PRIMARY REGION
8,products,REGIONAL BY TABLE IN PRIMARY REGION
9,region,REGIONAL BY TABLE IN PRIMARY REGION


---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

---

# Appendix

Workshop CRDB user id and passowrd

> <p>uid = roachie<br>
pwd = roachfan
</p>

List CRDB process id and process name.

> <code>pgrep -l cockroach</code>

List the listening address of each `cockroach` process.

> <code>pgrep -a cockroach | awk '{ print $5}'</code>

Kill ALL CRDB processes

> <code>kill -9  $(pgrep cockroach)</code>

Remove all CRDB files

> <code>sudo rm -fR /home/cockroach/data/*</code>

## Queries

### Reconnect to the IAM **database**

### List 5 users in the IAM database

In [ ]:
try:
    cursor.execute("""
      SELECT
          user_name
        , email
      FROM iam_users
      LIMIT 5
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set.style.hide(axis="index"))
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

user_name,email
lgethin0,kcamber0@google.com.au
spress1,labramamov1@cbc.ca
jhenric2,dbasset2@china.com.cn
acommin3,jjerrolt3@so-net.ne.jp
cpfeiffer4,nmumm4@goo.ne.jp


### Find all roles assigned to a specific user

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , role_name AS "Role"
FROM iam_roles INNER JOIN iam_users USING (role_id)
WHERE user_name = 'acommin3'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Role
acommin3,Auditor


### List all roles in the Keycloak database

In [ ]:
cursor.execute("""
SELECT role_name
FROM iam_roles
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

role_name
Admin/Super Admin
User Manager
Group Manager
Auditor
Helpdesk Support
Developer
External User


### Find all users with a specific role

In [ ]:
cursor.execute("""
SELECT user_name
FROM iam_users
WHERE user_id IN (
  SELECT user_id
  FROM iam_roles
  WHERE role_id = (
    SELECT role_id
    FROM iam_roles
    WHERE role_name = 'Admin/Super Admin'
  )
)
LIMIT 5
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

user_name
lgethin0
spress1
jhenric2
acommin3
cpfeiffer4


### Find all users assigned to a specific group
SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , log_timestamp AS "Log Entry Time"
  , activity AS "Activity Description"
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days'
LIMIT 10
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Log Entry Time,Activity Description
smaggoriniw,2024-07-16 12:05:58,Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis. Sed ante. Vivamus tortor. Duis mattis egestas metus. Aenean fermentum. Donec ut mauris eget massa tempor convallis.
bkliner1x,2023-11-16 19:35:59,Mauris lacinia sapien quis libero. Nullam sit amet turpis elementum ligula vehicula consequat. Morbi a ipsum.
bvatcher3m,2024-02-06 23:42:00,"Proin leo odio, porttitor id, consequat in, consequat ut, nulla. Sed accumsan felis."
bbegin41,2024-03-09 21:03:31,"Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl."
mrosettini4y,2024-08-29 22:18:49,Cras non velit nec nisi vulputate nonummy.
trothchild6d,2024-09-10 03:23:15,"Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus."
mpigginsej,2023-11-23 22:06:52,"Nam ultrices, libero non mattis pulvinar, nulla pede ullamcorper augue, a suscipit nulla elit ac nulla. Sed vel enim sit amet nunc viverra dapibus. Nulla suscipit ligula in lacus. Curabitur at ipsum ac tellus semper interdum. Mauris ullamcorper purus sit amet nulla. Quisque arcu libero, rutrum ac, lobortis vel, dapibus at, diam. Nam tristique tortor eu pede."
ikuschkego,2024-04-12 08:48:08,"Nunc nisl. Duis bibendum, felis sed interdum venenatis, turpis enim blandit mi, in porttitor pede justo eu massa. Donec dapibus. Duis at velit eu est congue elementum. In hac habitasse platea dictumst. Morbi vestibulum, velit id pretium iaculis, diam erat fermentum justo, nec condimentum neque sapien placerat ante. Nulla justo. Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis."
psandlandk6,2024-02-08 00:50:52,Proin risus.
bluppittrf,2024-06-28 00:00:57,Etiam faucibus cursus urna. Ut tellus. Nulla ut erat id mauris vulputate elementum. Nullam varius. Nulla facilisi. Cras non velit nec nisi vulputate nonummy. Maecenas tincidunt lacus at velit. Vivamus vel nulla eget eros elementum pellentesque.




-- Find all users assigned to a specific provider:
SELECT email, phone, provider_name
FROM iam_users INNER JOIN auth_provider USING(provider_id)
WHERE provider_type = 'Multi-factor Authentication provider';

-- Find all users assigned to a specific group: SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

-- Count the number of users in Keycloak:
SELECT COUNT(*) FROM iam_users;

-- Find all users who have not logged in for the past 30 days
SELECT user_name, log_timestamp
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days';